# Imports
All imports should be defined here to reduce clutter

In [ ]:
import random

import numpy as np
import pygame
import torch
import cv2
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import matplotlib.pyplot as plot
from novel_swarms.config.EvolutionaryConfig import GeneticEvolutionConfig
from novel_swarms.config.WorldConfig import RectangularWorldConfig
from novel_swarms.config.defaults import ConfigurationDefaults
from novel_swarms.novelty.GeneRule import GeneRule
from novel_swarms.config.OutputTensorConfig import OutputTensorConfig
from novel_swarms.novelty.NoveltyArchive import NoveltyArchive
from src.generation.halted_evolution import HaltedEvolution
from src.networks.encoder import BehaviorAutoEncoder

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Auto Encoder

In [ ]:
# Load a pre-trained AutoEncoder
encoder = BehaviorAutoEncoder()
encoder.load_model("cp_C-105")
encoder.to(device)
encoder.eval()

In [ ]:
import time
class NoveltySeparation(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.s1 = torch.nn.Sequential(
            torch.nn.Linear(256, 100),
            torch.nn.ReLU(),
            torch.nn.Linear(100, 60),
            torch.nn.Dropout(0.1),
            torch.nn.ReLU(),
            torch.nn.Linear(60, 20),
            torch.nn.ReLU(),
            torch.nn.Linear(20, 6),
        )

    def forward(self, x):
        x = self.s1(x)
        if not self.training:
            x = torch.nn.Softmax()(x)
        return x

    def load_model(self, file_name="cp_NAME"):
        checkpoint = torch.load(f"checkpoints/mixer/{file_name}.pt")
        self.load_state_dict(checkpoint["model_state_dict"])

    def save_model(self):
        file_name = f"cp_{round(time.time())}"
        torch.save({
            'model_state_dict': self.state_dict(),
        }, f"checkpoints/mixer/{file_name}.pt")

# Generator

In [ ]:
def getGenerator(steps=1200):
    agent_config = ConfigurationDefaults.DIFF_DRIVE_AGENT

    genotype = [
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
    ]

    phenotype = ConfigurationDefaults.BEHAVIOR_VECTOR

    world_config = RectangularWorldConfig(
        size=(500, 500),
        n_agents=30,
        behavior=phenotype,
        agentConfig=agent_config,
        padding=15
    )

    novelty_config = GeneticEvolutionConfig(
        gene_rules=genotype,
        phenotype_config=phenotype,
        n_generations=100,
        n_population=100,
        crossover_rate=0.7,
        mutation_rate=0.15,
        world_config=world_config,
        k_nn=15,
        simulation_lifespan=steps,
        display_novelty=False,
        save_archive=False,
        show_gui=True
    )

    pygame.init()
    pygame.display.set_caption("Evolutionary Novelty Search")
    screen = pygame.display.set_mode((world_config.w, world_config.h))

    output_config = OutputTensorConfig(
        timeless=True,
        total_frames=80,
        steps_between_frames=2,
        screen=screen
    )

    halted_evolution = HaltedEvolution(
        world=world_config,
        evolution_config=novelty_config,
        output_config=output_config
    )

    return halted_evolution, screen

def shakeGenome(genome):
    return genome + (np.random.rand(len(genome)) * 0.01)

def antiGenome(genome):
    return -1 * shakeGenome(genome)

def resizeInput(X):
    frame = X.astype(np.uint8)
    resized = cv2.resize(frame, dsize=(200, 200), interpolation=cv2.INTER_CUBIC)
    return resized

def imgToEncoded(X):
    X = torch.Tensor(resizeInput(X)).to(device)
    X = torch.reshape(X, (1, 200*200))
    f_X = encoder.encoded(X).to(device)
    return X, f_X

def weights_init_xavier_rule(m):
    if type(m) == torch.nn.Linear:
        torch.nn.init.xavier_normal_(m.weight)


In [ ]:
K = 10
generator, screen = HaltedEvolution.defaultEvolver(steps=1200, evolve_population=20, k_samples=K)
encodedArchive = NoveltyArchive(max_size=1000)
noveltyClassifier = NoveltySeparation().to(device)
noveltyClassifier.apply(weights_init_xavier_rule)
noveltyClassifier.train()

writer = SummaryWriter()

optimizer = torch.optim.Adam(noveltyClassifier.parameters(), lr=1e-2)
loss_fn = torch.nn.TripletMarginLoss(margin=20)
generation_size = generator.evolve_config.generations
population_size = generator.evolve_config.population

for generation in range(generation_size):

    img_behaviors = []
    behavior_vectors = []
    b_novelty = []
    f_novelty = []
    genomes = []
    encodings = []
    sortable_scores = []
    blended_scores = [torch.Tensor([0.0]) for _ in range(population_size)]

    total_loss = 0.0
    batch_anchors = torch.Tensor([]).to(device)
    batch_positives = torch.Tensor([]).to(device)
    batch_negatives = torch.Tensor([]).to(device)
    BATCH_SIZE = 2

    for i in range(population_size):
        optimizer.zero_grad()
        X, b, genome = generator.next()
        genomes.append(genome)
        behavior_vectors.append(b)
        img_behaviors.append(X.copy())

        X, f_X = imgToEncoded(X)
        b = torch.Tensor(b).to(device)
        encoding = f_X.detach().cpu().squeeze(0).numpy()
        encodings.append(encoding)
        encodedArchive.addToArchive(encoding, genome)

        inp = torch.concat((f_X.squeeze(0), b))
        anchor_out = noveltyClassifier(inp)
        batch_anchors = torch.concat((batch_anchors, anchor_out.unsqueeze(0)))

        positive = shakeGenome(genome)
        positive, b = generator.simulation(positive)
        b = torch.Tensor(b).to(device)
        positive, pos_f_x = imgToEncoded(positive)
        inp = torch.concat((pos_f_x.squeeze(0), b))
        pos_out = noveltyClassifier(inp)
        batch_positives = torch.concat((batch_positives, pos_out.unsqueeze(0)))

        negative = antiGenome(genome)
        negative, b = generator.simulation(negative)
        b = torch.Tensor(b).to(device)
        negative, neg_f_x = imgToEncoded(negative)
        inp = torch.concat((neg_f_x.squeeze(0), b))
        neg_out = noveltyClassifier(inp)
        batch_negatives = torch.concat((batch_negatives, neg_out.unsqueeze(0)))

        print("Anchor: ", anchor_out)
        print("Positive: ", pos_out)
        print("Negative: ", neg_out)

        loss = loss_fn(anchor_out, pos_out, neg_out)
        loss.backward()
        total_loss += loss.item()
        print("Batch Loss: ", loss.item())
        optimizer.step()

        # if (i + 1) % BATCH_SIZE == 0:
        #     loss = loss_fn(batch_anchors, batch_positives, batch_negatives)
        #     loss.backward()
        #     total_loss += loss.item()
        #     print("Batch Loss: ", loss.item())
        #     optimizer.step()
        #
        #     batch_anchors = torch.Tensor([]).to(device)
        #     batch_positives = torch.Tensor([]).to(device)
        #     batch_negatives = torch.Tensor([]).to(device)


    print(f"Generation: {generation}, Training Loss: {total_loss}")
    writer.add_scalar("training/loss", total_loss / population_size, generation)
    generator.behavior_discovery.evaluate()
    print(generator.behavior_discovery.scores)

    # for elem, encoded_b in enumerate(encodings):
    #     encoded_novelty = encodedArchive.getNovelty(K, encoded_b)
    #     f_novelty.append(encoded_novelty)
    #     behavior_novelty = generator.behavior_discovery.scores[elem]
    #     b_novelty.append(behavior_novelty)
    #     blend_out = noveltyBlender(torch.Tensor([encoded_novelty, behavior_novelty]))
    #     blended_scores[elem] = blend_out
    #     sortable_scores.append((blend_out, elem))

    generator.behavior_discovery.scores = blended_scores

    HIL_SAMPLES = 1
    # if generation % 3 == 0:
    #     for sample in range(HIL_SAMPLES):
    #         # Human in the Loop
    #         # Present the user with the top three images and ask them to rank them in terms of novelty
    #         split = population_size // 3
    #         a_index = random.randint(0, split - 1)
    #         b_index = random.randint(split, split * 2 - 1)
    #         c_index = random.randint(split*2, split * 3 - 1)
    #
    #         print("Random index selections: ", a_index, b_index, c_index)
    #         a_index = sortable_scores[a_index][1]
    #         b_index = sortable_scores[b_index][1]
    #         c_index = sortable_scores[c_index][1]
    #
    #         fig, (ax1, ax2, ax3) = plot.subplots(1, 3)
    #         ax1.imshow(img_behaviors[a_index], cmap="Greys")
    #         ax1.set_title("Sample A")
    #         ax2.imshow(img_behaviors[b_index], cmap="Greys")
    #         ax2.set_title("Sample B")
    #         ax3.imshow(img_behaviors[c_index], cmap="Greys")
    #         ax3.set_title("Sample C")
    #         plot.pause(2)
    #
    #         responses = {
    #             "A" : a_index,
    #             "B" : b_index,
    #             "C" : c_index,
    #         }
    #
    #         # Input A,B,C in ranked order in the form "ABC"
    #         time.sleep(0.2)
    #         user_res = ""
    #         while len(user_res) != 1 or not user_res.capitalize() in responses:
    #             user_res = input("Which one of these is least like the others?")
    #             if user_res == "q":
    #                 break
    #             if user_res == "w":
    #                 time.sleep(3)

            # different_index = responses[user_res.capitalize()]
            # res_to_same = list(responses.keys())
            # res_to_same.remove(user_res.capitalize())
            # same_indices = [responses[i] for i in res_to_same]
            # assert len(same_indices) == 2
            #
            # anchor_out = noveltyClassifier(torch.Tensor([f_novelty[same_indices[0]], b_novelty[same_indices[0]]]))
            # pos_out = noveltyClassifier(torch.Tensor([f_novelty[same_indices[1]], b_novelty[same_indices[1]]]))
            # neg_out = noveltyClassifier(torch.Tensor([f_novelty[different_index], b_novelty[different_index]]))
            #
            # optimizer.zero_grad()
            # loss = loss_fn(anchor_out, pos_out, neg_out)
            # loss.backward()
            # total_loss += loss.item()
            # optimizer.step()



pygame.quit()

In [ ]:
noveltyClassifier.save_model()

In [ ]:
# if True:
#     noveltyClassifier = NoveltySeparation()
#     noveltyClassifier.load_model("cp_t1_5e_6o")
#     noveltyClassifier.to(device)

noveltyClassifier.eval()
generator, screen = HaltedEvolution.defaultEvolver(steps=1200, evolve_population=50, k_samples=12)
SAMPLES = 5

for i in range(SAMPLES):
    cyclic = [-0.7, 0.3, 1.0, 1.0]
    agg = [-0.7, -1.0, 1.0, -1.0]
    dispersal = [0.2, 0.7, -0.5, -0.1]
    milling = [-0.69, -0.77, 0.05, -0.4]
    wall_f = [1.0, 0.98, 1.0, 1.0]
    random = [-0.83889, -0.7501, 0.27992, -0.57196]

    img, b = generator.simulation(random)
    b = torch.Tensor(b).to(device)
    encoded, neg_f_x = imgToEncoded(img)
    inp = torch.concat((neg_f_x.squeeze(0), b))
    output = noveltyClassifier(inp)
    class_guess = torch.argmax(output)
    print("Class Guess: ", class_guess.item())

    img, b = generator.simulation(antiGenome(dispersal))
    # img, b = generator.simulation(antiGenome(agg))
    b = torch.Tensor(b).to(device)
    encoded, neg_f_x = imgToEncoded(img)
    inp = torch.concat((neg_f_x.squeeze(0), b))
    output = noveltyClassifier(inp)
    class_guess = torch.argmax(output)
    print("Anti Guess: ", class_guess.item())

# plot.imshow(img, cmap="Greys")
print(output)
pygame.quit()